In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from scipy.spatial import Voronoi, voronoi_plot_2d
from shapely.geometry import Polygon, Point
from shapely.ops import unary_union
import mplcursors

In [ ]:
players = pd.read_csv("players.csv")
plays = pd.read_csv("plays.csv")
playerplays = pd.read_csv("player_play.csv")

In [ ]:
weekOneTracking = pd.read_csv("tracking_week_1.csv")
weekTwoTracking = pd.read_csv("tracking_week_2.csv")
weekThreeTracking = pd.read_csv("tracking_week_3.csv")
weekFourTracking = pd.read_csv("tracking_week_4.csv")
weekFiveTracking = pd.read_csv("tracking_week_5.csv")
weekSixTracking = pd.read_csv("tracking_week_6.csv")
weekSevenTracking = pd.read_csv("tracking_week_7.csv")
weekEightTracking = pd.read_csv("tracking_week_8.csv")
weekNineTracking = pd.read_csv("tracking_week_9.csv")

In [ ]:
weekOneTracking = pd.merge(
    weekOneTracking,
    players[['nflId','position']],
    how = "left",
    on = "nflId"
)
weekTwoTracking = pd.merge(
    weekTwoTracking,
    players[['nflId','position']],
    how = "left",
    left_on=['nflId'],
    right_on=['nflId']
)
weekThreeTracking = pd.merge(
    weekThreeTracking,
    players[['nflId','position']],
    how = "left",
    left_on=['nflId'],
    right_on=['nflId']
)
weekFourTracking = pd.merge(
    weekFourTracking,
    players[['nflId','position']],
    how = "left",
    left_on=['nflId'],
    right_on=['nflId']
)
weekFiveTracking = pd.merge(
    weekFiveTracking,
    players[['nflId','position']],
    how = "left",
    left_on=['nflId'],
    right_on=['nflId']
)
weekSixTracking = pd.merge(
    weekSixTracking,
    players[['nflId','position']],
    how = "left",
    left_on=['nflId'],
    right_on=['nflId']
)
weekSevenTracking = pd.merge(
    weekSevenTracking,
    players[['nflId','position']],
    how = "left",
    left_on=['nflId'],
    right_on=['nflId']
)
weekEightTracking = pd.merge(
    weekEightTracking,
    players[['nflId','position']],
    how = "left",
    left_on=['nflId'],
    right_on=['nflId']
)
weekNineTracking = pd.merge(
    weekNineTracking,
    players[['nflId','position']],
    how = "left",
    left_on=['nflId'],
    right_on=['nflId']
)

In [ ]:
def clean_data_voronoi(sampleLoop):
    sampleLoop = sampleLoop[(sampleLoop['displayName']!="football")]
    sampleLoop = sampleLoop[(sampleLoop['frameType'] == "AFTER_SNAP") | (sampleLoop['frameType']=="SNAP")]
    sampleLoop = pd.merge(
        sampleLoop,
        plays[['gameId','playId','timeToThrow','timeToSack','pff_manZone', 'possessionTeam', 'yardlineSide', 'absoluteYardlineNumber','passResult','yardlineNumber']],
        how = "left",
        left_on = ['gameId','playId'],
        right_on=['gameId','playId']
    )
    sampleLoop = sampleLoop[sampleLoop['passResult'].notna()]
    snap_frames = sampleLoop[sampleLoop['frameType'] == 'SNAP'][['gameId', 'playId', 'frameId']]
    sampleLoop = pd.merge(sampleLoop, snap_frames, on=['gameId', 'playId'], how='left', suffixes=('', '_snapframe'))
    sampleLoop['snapframe'] = sampleLoop['frameId_snapframe']
    after_snap_data = sampleLoop.drop(columns=['frameId_snapframe'])
    timeSinceSnap = (after_snap_data['frameId']-after_snap_data['snapframe'])/10
    after_snap_data = after_snap_data[(timeSinceSnap > 1)&(timeSinceSnap<after_snap_data['timeToThrow'])]
    return after_snap_data

In [ ]:
def max_voronoi_per_play(after_snap_data):
    results = []
    for name, group in after_snap_data.groupby(['gameId', 'playId', 'frameId']):
        frame = group[['nflId','club','x','y','position','gameId','playId','frameId','timeToThrow','snapframe','possessionTeam', 'yardlineSide','absoluteYardlineNumber','yardlineNumber']]
        frame['timeSinceSnap'] = (frame['frameId'] - frame['snapframe'])/10
        new_rows = []
        for _, row in frame.iterrows():
            if row['position'] in ['WR', 'TE', 'RB']:
                if row['possessionTeam'] == row['yardlineSide']:
                    if row['absoluteYardlineNumber'] < 60:
                        xpoint = row['x'] - 5
                    else:
                        xpoint = row['x'] + 5
                else:
                    if row['absoluteYardlineNumber'] > 60:
                        xpoint = row['x'] - 5
                    else:
                        xpoint = row['x'] + 5  
                new_row = {
                    'x': xpoint,
                    'y': row['y'],
                    'position': '',
                    'club': None,
                    'nflId': 0,
                    'gameId': row['gameId'],
                    'playId': row['playId'],
                    'frameId': row['frameId']
                }
                new_rows.append(new_row)
        new_rows_df = pd.DataFrame(new_rows)
        newFrames = pd.concat([frame, new_rows_df], ignore_index=True)
        points = newFrames[['x', 'y']].values
        x_min, x_max = 0, 120
        y_min, y_max = 0, 53.3
        boundary_points = np.array([
            [x_min, y_min],
            [x_min, y_max],
            [x_max, y_min],
            [x_max, y_max]
        ])
        all_points = np.vstack([points, boundary_points])
        vor = Voronoi(all_points)
        boundary_polygon = Polygon([(x_min, y_min), (x_min, y_max), (x_max, y_max), (x_max, y_min)])
        
        clipped_polygons = []
        polygon_areas = []
        for region in vor.regions:
            if not -1 in region and len(region) > 0:
                polygon = Polygon([vor.vertices[i] for i in region])
                clipped_polygon = polygon.intersection(boundary_polygon)
                if not clipped_polygon.is_empty:
                    clipped_polygons.append(clipped_polygon)
                    polygon_areas.append(clipped_polygon.area)

        player_info = {tuple(row[['x', 'y']].values): {'nflId': row['nflId'], 'position': row['position'], 'TTT': row['timeToThrow'], 'timeSinceSnap': row['timeSinceSnap'], 'yardline':row['x'],'absoluteYardLine':row['absoluteYardlineNumber'], 'playYardLine':row['yardlineNumber'], 'possessionTeam':row['possessionTeam'], 'yardlineSide':row['yardlineSide']} 
                    for idx, row in newFrames.iterrows()}
        for poly, area in zip(clipped_polygons, polygon_areas):
            for point_coords, info in player_info.items():
                point = Point(point_coords)
                if poly.contains(point) and info['position'] in ['TE','RB','WR']:
                    if info['possessionTeam'] == info['yardlineSide']:
                        if info['absoluteYardLine'] < 60:
                            downfield = info['yardline'] - info['absoluteYardLine']
                        else:
                            downfield = info['absoluteYardLine'] - info['yardline']
                    else:
                        if info['absoluteYardLine'] > 60:
                            downfield = info['yardline'] - info['absoluteYardLine']
                        else:
                            downfield = info['absoluteYardLine'] - info['yardline']
                    results.append({
                        'gameId': name[0],
                        'playId': name[1],
                        'frameId': name[2],
                        'nflId': info['nflId'],
                        'position': info['position'],
                        'voronoiArea': area,
                        'timeToThrow': info['TTT'],
                        'timeSinceSnap': info['timeSinceSnap'],
                        'yardsDownField': downfield
                    })
    results = pd.DataFrame(results)
    results['valueOfSeparation'] = (results['voronoiArea'] * (1.01750249 ** results['yardsDownField']))
    return results

In [ ]:
def group_voronoi_results(results):
    max_separation_per_play = results.groupby(['gameId', 'playId', 'nflId'])['valueOfSeparation'].max().reset_index()
    max_separation_per_play.columns = ['gameId', 'playId', 'routeRunnerId', 'maxValueOfSeparation']
    
    result = pd.merge(
        max_separation_per_play,
        results[['gameId', 'playId', 'nflId', 'frameId', 'valueOfSeparation', 'timeSinceSnap', 'timeToThrow', 'yardsDownField']],  # Select necessary columns
        how='left',
        left_on=['gameId', 'playId', 'routeRunnerId', 'maxValueOfSeparation'],  # Join on the maximum separation
        right_on=['gameId', 'playId', 'nflId', 'valueOfSeparation']  # Join on the separation value
    )
    result = result.drop(columns=['nflId','valueOfSeparation'])
    return result

In [ ]:
motionplays = pd.read_csv("allWeekMotionPlays")
motionindex = motionplays.iloc[:,1:3]
motion_index_pairs = motionindex[['gameId', 'playId']].drop_duplicates()

In [ ]:
filteredWeekOne = weekOneTracking.merge(motion_index_pairs, on=['gameId', 'playId'], how='inner')
filteredWeekTwo = weekTwoTracking.merge(motion_index_pairs, on=['gameId', 'playId'], how='inner')
filteredWeekThree = weekThreeTracking.merge(motion_index_pairs, on=['gameId', 'playId'], how='inner')
filteredWeekFour = weekFourTracking.merge(motion_index_pairs, on=['gameId', 'playId'], how='inner')
filteredWeekFive = weekFiveTracking.merge(motion_index_pairs, on=['gameId', 'playId'], how='inner')
filteredWeekSix = weekSixTracking.merge(motion_index_pairs, on=['gameId', 'playId'], how='inner')
filteredWeekSeven = weekSevenTracking.merge(motion_index_pairs, on=['gameId', 'playId'], how='inner')
filteredWeekNine = weekNineTracking.merge(motion_index_pairs, on=['gameId', 'playId'], how='inner')

In [ ]:
sample = filteredWeekOne
cleanedSample = clean_data_voronoi(sample)
voronoiCalc = max_voronoi_per_play(cleanedSample)
groupedVoronoi = group_voronoi_results(voronoiCalc)
groupedVoronoi.to_csv("voronoi1.csv")
sample2 = filteredWeekTwo
cleanedSample2 = clean_data_voronoi(sample2)
voronoiCalc2 = max_voronoi_per_play(cleanedSample2)
groupedVoronoi2 = group_voronoi_results(voronoiCalc2)
groupedVoronoi2.to_csv("voronoi2.csv")
sample3 = filteredWeekThree
cleanedSample3 = clean_data_voronoi(sample3)
voronoiCalc3 = max_voronoi_per_play(cleanedSample3)
groupedVoronoi3 = group_voronoi_results(voronoiCalc3)
groupedVoronoi3.to_csv("voronoi3.csv")
sample4 = filteredWeekFour
cleanedSample4 = clean_data_voronoi(sample4)
voronoiCalc4 = max_voronoi_per_play(cleanedSample4)
groupedVoronoi4 = group_voronoi_results(voronoiCalc4)
groupedVoronoi4.to_csv("voronoi4.csv")
sample5 = filteredWeekFive
cleanedSample5 = clean_data_voronoi(sample5)
cleanedSample5
voronoiCalc5 = max_voronoi_per_play(cleanedSample5)
groupedVoronoi5 = group_voronoi_results(voronoiCalc5)
groupedVoronoi5.to_csv("voronoi5.csv")
sample6 = filteredWeekSix
cleanedSample6 = clean_data_voronoi(sample6)
voronoiCalc6 = max_voronoi_per_play(cleanedSample6)
groupedVoronoi6 = group_voronoi_results(voronoiCalc6)
groupedVoronoi6.to_csv("voronoi6.csv")
sample7 = filteredWeekSeven
cleanedSample7 = clean_data_voronoi(sample7)
voronoiCalc7= max_voronoi_per_play(cleanedSample7)
groupedVoronoi7 = group_voronoi_results(voronoiCalc7)
groupedVoronoi7.to_csv("voronoi7.csv")
sample9 = filteredWeekNine
cleanedSample9 = clean_data_voronoi(sample9)
voronoiCalc9 = max_voronoi_per_play(cleanedSample9)
groupedVoronoi9 = group_voronoi_results(voronoiCalc9)
groupedVoronoi9.to_csv("voronoi9.csv")